In [1]:
# Import librairies

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import plotly.figure_factory as ff

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
# URL of the CSV file of INSEE data
insee_url = 'https://medical-deserts-project.s3.eu-north-1.amazonaws.com/insee_clean.csv'

# Read the CSV file from the URL into a DataFrame
insee_df_original = pd.read_csv(insee_url, sep = ',', encoding='utf-8')
insee_df_original.shape

(38590, 90)

In [4]:
insee_df = insee_df_original.copy()

In [5]:
# Remove useless columns
insee_df = insee_df.drop(["APL aux médecins généralistes de 65 ans et moins", "APL aux médecins généralistes de 62 ans et moins"], axis=1)

# APL column at the end of dataset
insee_df["APL aux médecins généralistes (sans borne d'âge)"] = insee_df.pop("APL aux médecins généralistes (sans borne d'âge)")
insee_df.rename(columns={"APL aux médecins généralistes (sans borne d'âge)": "APL"}, inplace=True)
insee_df.shape

(38590, 88)

In [6]:
insee_df.drop_duplicates(inplace = True)

In [7]:
to_drop = ["Nb pharmaciens Libéraux BV" ,"Nb Entreprises Secteur Services", "Nb Entreprises Secteur Commerce", "Nb Ménages", "Nb Résidences Principales", "Nb Occupants Résidence Principale", "Nb Création Commerces", "Nb Création Enteprises", "PIB Régionnal", "Nb de Commerce", "Nb Santé, action sociale", "Population en 2014 (princ)", "Pop 60-74 ans en 2014 (princ)", "Pop 75-89 ans en 2014 (princ)", "Nb Logement Secondaire et Occasionnel"]

# Remove columns to be dropped
insee_df = insee_df.drop(columns=to_drop)

In [8]:
insee_df.shape

(34760, 73)

## RF avec les meilleurs hyperparamètres sur toutes les features 

In [9]:
# X, y split 
X = insee_df.loc[:, insee_df.columns != "APL"]
y = insee_df.loc[:, "APL"]

In [10]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Dynamique Entrepreneuriale', 'Dynamique Entrepreneuriale Service et Commerce', 'Synergie Médicale COMMUNE', 'Nb Omnipraticiens BV', 'Nb Infirmiers Libéraux BV', 'Nb dentistes Libéraux BV', 'Densité Médicale BV', 'Score équipement de santé BV', 'Indice Démographique', 'Nb propriétaire', 'Nb Logement', 'Nb Résidences Secondaires', 'Nb Log Vacants', 'Nb Entreprises Secteur Construction', 'Nb Entreprises Secteur Industrie', 'Nb Création Industrielles', 'Nb Création Construction', 'Nb Création Services', 'Moyenne Revenus Fiscaux Départementaux', 'Moyenne Revenus Fiscaux Régionaux', 'Dep Moyenne Salaires Horaires', 'Dep Moyenne Salaires Cadre Horaires', 'Dep Moyenne Salaires Prof Intermédiaire Horaires', 'Dep Moyenne Salaires Employé Horaires', 'Dep Moyenne Salaires Ouvrié Horaires', 'Reg Moyenne Salaires Horaires', 'Reg Moyenne Salaires Cadre Horaires', 'Reg Moyenne Salaires Prof Intermédiaire Horaires', 'Reg Moyenne Salaires Employé Horaires', 'Reg Moyenne Salaire

In [11]:
# Train_test_split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)


In [12]:
#categorical_transformer = OneHotEncoder(drop='first')
from sklearn.pipeline import Pipeline

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))])

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [13]:
# Fit and transform X_train
X_train = preprocessor.fit_transform(X_train)
# Apply on X_test
X_test = preprocessor.transform(X_test)

# Visualize X_std_train
X_train

array([[-0.02231907,  0.0064759 , -0.00263147, ...,  0.        ,
         1.        ,  0.        ],
       [-0.19154912, -0.15064951, -0.09054271, ...,  0.        ,
         0.        ,  1.        ],
       [-0.202126  , -0.148864  , -0.25284038, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.14426677, -0.1274378 ,  0.95762974, ...,  0.        ,
         1.        ,  0.        ],
       [-0.16775114, -0.13100884,  0.18671581, ...,  0.        ,
         1.        ,  0.        ],
       [-0.18626068, -0.13993642, -0.3069396 , ...,  0.        ,
         1.        ,  0.        ]])

In [14]:
rf = RandomForestRegressor(max_depth = 18, min_samples_leaf = 2, min_samples_split= 2, n_estimators= 80)

In [15]:
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=18, min_samples_leaf=2, n_estimators=80)

In [16]:
# Print R^2 scores
print("R2 score on training set : ", rf.score(X_train, y_train))
print("R2 score on test set : ", rf.score(X_test, y_test))

R2 score on training set :  0.8030395862731291
R2 score on test set :  0.4021296013439084


In [17]:
column_names = []
for name, step, features_list in preprocessor.transformers_: # loop over steps of ColumnTransformer
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = step.get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Dynamique Entrepreneuriale', 'Dynamique Entrepreneuriale Service et Commerce', 'Synergie Médicale COMMUNE', 'Nb Omnipraticiens BV', 'Nb Infirmiers Libéraux BV', 'Nb dentistes Libéraux BV', 'Densité Médicale BV', 'Score équipement de santé BV', 'Indice Démographique', 'Nb propriétaire', 'Nb Logement', 'Nb Résidences Secondaires', 'Nb Log Vacants', 'Nb Entreprises Secteur Construction', 'Nb Entreprises Secteur Industrie', 'Nb Création Industrielles', 'Nb Création Construction', 'Nb Création Services', 'Moyenne Revenus Fiscaux Départementaux', 'Moyenne Revenus Fiscaux Régionaux', 'Dep Moyenne Salaires Horaires', 'Dep Moyenne Salaires Cadre Horaires', 'Dep Moyenne Salaires Prof Intermédiaire Horaires', 'Dep Moyenne Salaires Employé Horaires', 'Dep Moyenne Salaires Ouvrié Horaires', 'Reg Moyenne Salaires Horaires', 'Reg Moyenne Salaires Cadre Horaires', 'Reg Moyenne Salaires Prof Intermédiaire Horaires', 'Reg Moyenne Salaires Employé Ho

In [18]:
feature_importance = pd.DataFrame(index = column_names, data = rf.feature_importances_, columns=["feature_importances"])
feature_importance = feature_importance.sort_values(by = 'feature_importances')

In [19]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(height=2000, width=2500, showlegend = False, margin = {'l': 50}, ) # to avoid cropping of column names
fig.show()

In [26]:
feature_importance.sort_values(by = 'feature_importances', ascending = False, inplace = True)

In [27]:
best30_features = feature_importance[:30]

In [29]:
best30_features = best30_features.index

In [32]:
best30_features = [x.split('_')[0] for x in best30_features]

In [33]:
best30_features

['Dynamique Entrepreneuriale Service et Commerce',
 'latitude',
 'longitude',
 'Taux Propriété',
 'Densité Médicale BV',
 'Nb Résidences Secondaires',
 'taux chômage(15-64 ans)',
 'Synergie Médicale COMMUNE',
 'Pop 15 ans ou plus Prof. intermédiaires  en 2014 (compl)',
 'Pop 15 ans ou plus Retraités  en 2014 (compl)',
 'Pop 15 ans ou plus Employés en 2014 (compl)',
 'Nb Log Vacants',
 'Dynamique Entrepreneuriale',
 'Pop 0-14 ans en 2014 (princ)',
 'Pop 15 ans ou plus Ouvriers en 2014 (compl)',
 'Pop 30-44 ans en 2014 (princ)',
 'Pop 15 ans ou plus Cadres, Prof. intel. sup. en 2014 (compl)',
 'Capacité Fisc',
 'Nb Omnipraticiens BV',
 'Dep Moyenne Salaires Employé Horaires',
 'Pop 15 ans ou plus Agriculteurs exploitants en 2014 (compl)',
 'Pop 45-59 ans en 2014 (princ)',
 'Pop 15-29 ans en 2014 (princ)',
 'Reg Moyenne Salaires Employé Horaires',
 'Nb Infirmiers Libéraux BV',
 'Pop 15 ans ou plus Autres en 2014 (compl)',
 'Pop 15 ans ou plus Artisans, Comm., Chefs entr. en 2014 (compl)',

## RF avec les best features de claudine 

In [36]:
# X, y split 

X = insee_df.loc[:, best30_features]
y = insee_df.loc[:, "APL"]

# X = X.select_dtypes(exclude=["object"])


#Avec les best features de claudine : 
# Train_test_split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)


In [37]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Dynamique Entrepreneuriale Service et Commerce', 'latitude', 'longitude', 'Taux Propriété', 'Densité Médicale BV', 'Nb Résidences Secondaires', 'taux chômage(15-64 ans)', 'Synergie Médicale COMMUNE', 'Pop 15 ans ou plus Prof. intermédiaires  en 2014 (compl)', 'Pop 15 ans ou plus Retraités  en 2014 (compl)', 'Pop 15 ans ou plus Employés en 2014 (compl)', 'Nb Log Vacants', 'Dynamique Entrepreneuriale', 'Pop 0-14 ans en 2014 (princ)', 'Pop 15 ans ou plus Ouvriers en 2014 (compl)', 'Pop 30-44 ans en 2014 (princ)', 'Pop 15 ans ou plus Cadres, Prof. intel. sup. en 2014 (compl)', 'Capacité Fisc', 'Nb Omnipraticiens BV', 'Dep Moyenne Salaires Employé Horaires', 'Pop 15 ans ou plus Agriculteurs exploitants en 2014 (compl)', 'Pop 45-59 ans en 2014 (princ)', 'Pop 15-29 ans en 2014 (princ)', 'Reg Moyenne Salaires Employé Horaires', 'Nb Infirmiers Libéraux BV', 'Pop 15 ans ou plus Autres en 2014 (compl)', 'Pop 15 ans ou plus Artisans, Comm., Chefs entr. en 2014 (compl)', '

In [38]:
#categorical_transformer = OneHotEncoder(drop='first')
from sklearn.pipeline import Pipeline

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))])

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [39]:
# Train_test_split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)


In [40]:
# Fit and transform X_train
X_train = preprocessor.fit_transform(X_train)
# Apply on X_test
X_test = preprocessor.transform(X_test)

# Visualize X_std_train
X_train

array([[ 0.0064759 , -1.40621685, -1.57839888, ...,  0.        ,
         1.        ,  0.        ],
       [-0.15064951,  1.00360727, -0.96135134, ...,  0.        ,
         1.        ,  0.        ],
       [-0.148864  ,  0.71634156, -1.0928393 , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.1274378 ,  0.77616326,  0.0775988 , ...,  0.        ,
         1.        ,  0.        ],
       [-0.13100884,  0.7802653 ,  1.64548557, ...,  0.        ,
         1.        ,  0.        ],
       [-0.13993642,  1.06605536,  0.66664637, ...,  0.        ,
         1.        ,  0.        ]])

In [41]:
# Instanciate RandomForestRegressor
rf = RandomForestRegressor(max_depth = 18, min_samples_leaf = 2, min_samples_split=2, n_estimators= 80)

In [42]:
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=18, min_samples_leaf=2, n_estimators=80)

In [43]:
# Print R^2 scores
print("R2 score on training set : ", rf.score(X_train, y_train))
print("R2 score on test set : ", rf.score(X_test, y_test))

R2 score on training set :  0.7997839247587135
R2 score on test set :  0.3813396666754284


Score RF (max_depth=18, min_samples_leaf=2, n_estimators=80) on 30 best features:

R2 score on training set :  0.7997839247587135

R2 score on test set :  0.3813396666754284

# Grid search Random Forest on 30 best features : 

In [44]:
# Perform grid search
rf = RandomForestRegressor()
#max_depth = 14, min_samples_leaf = 2, min_samples_split= 2, n_estimators = 80)

print("Grid search...")

# Grid of values to be tested
params = {
     'max_depth': [16, 18, 20],
     'min_samples_split': [1, 2, 4],
     'n_estimators': [60, 80, 100],
     'min_samples_leaf': [1, 2, 4] }

gridsearch = GridSearchCV(rf, param_grid= params, cv = 3, verbose = 2) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, y_train)
print("...Done.")

print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation accuracy : ", gridsearch.best_score_)


Grid search...
Fitting 3 folds for each of 81 candidates, totalling 243 fits
[CV] END max_depth=16, min_samples_leaf=1, min_samples_split=1, n_estimators=60; total time=   0.0s
[CV] END max_depth=16, min_samples_leaf=1, min_samples_split=1, n_estimators=60; total time=   0.0s
[CV] END max_depth=16, min_samples_leaf=1, min_samples_split=1, n_estimators=60; total time=   0.0s
[CV] END max_depth=16, min_samples_leaf=1, min_samples_split=1, n_estimators=80; total time=   0.0s
[CV] END max_depth=16, min_samples_leaf=1, min_samples_split=1, n_estimators=80; total time=   0.0s
[CV] END max_depth=16, min_samples_leaf=1, min_samples_split=1, n_estimators=80; total time=   0.0s
[CV] END max_depth=16, min_samples_leaf=1, min_samples_split=1, n_estimators=100; total time=   0.0s
[CV] END max_depth=16, min_samples_leaf=1, min_samples_split=1, n_estimators=100; total time=   0.0s
[CV] END max_depth=16, min_samples_leaf=1, min_samples_split=1, n_estimators=100; total time=   0.0s
[CV] END max_depth=1

c:\Users\utilisateur\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning:


81 fits failed out of a total of 243.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\utilisateur\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\utilisateur\anaconda3\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\utilisateur\anaconda3\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\utilisat

...Done.
Best hyperparameters :  {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 100}
Best validation accuracy :  0.34104323151527427


In [45]:
print("R2 score on training set : ", gridsearch.score(X_train, y_train))
print("R2 score on test set : ",     gridsearch.score(X_test, y_test))

R2 score on training set :  0.8374176267961229
R2 score on test set :  0.3850203644448976


In [46]:
column_names = []
for name, step, features_list in preprocessor.transformers_: # loop over steps of ColumnTransformer
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = step.get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Dynamique Entrepreneuriale Service et Commerce', 'latitude', 'longitude', 'Taux Propriété', 'Densité Médicale BV', 'Nb Résidences Secondaires', 'taux chômage(15-64 ans)', 'Synergie Médicale COMMUNE', 'Pop 15 ans ou plus Prof. intermédiaires  en 2014 (compl)', 'Pop 15 ans ou plus Retraités  en 2014 (compl)', 'Pop 15 ans ou plus Employés en 2014 (compl)', 'Nb Log Vacants', 'Dynamique Entrepreneuriale', 'Pop 0-14 ans en 2014 (princ)', 'Pop 15 ans ou plus Ouvriers en 2014 (compl)', 'Pop 30-44 ans en 2014 (princ)', 'Pop 15 ans ou plus Cadres, Prof. intel. sup. en 2014 (compl)', 'Capacité Fisc', 'Nb Omnipraticiens BV', 'Dep Moyenne Salaires Employé Horaires', 'Pop 15 ans ou plus Agriculteurs exploitants en 2014 (compl)', 'Pop 45-59 ans en 2014 (princ)', 'Pop 15-29 ans en 2014 (princ)', 'Reg Moyenne Salaires Employé Horaires', 'Nb Infirmiers Libéraux BV', 'Pop 15 ans ou plus Autres en 2014 (compl)', 'Pop 15 ans ou plus Artisans, Comm., Ch

In [57]:
# Create a pandas DataFrame
feature_importance = pd.DataFrame(index = column_names, data = gridsearch.best_estimator_.feature_importances_, columns=["feature_importances"])
feature_importance = feature_importance.sort_values(by = 'feature_importances')

In [58]:
feature_importance

,feature_importances
Urbanité Ruralité_Com rurale > 2 000 habts,0.000349
Urbanité Ruralité_Com < 50 m habts,0.000976
Urbanité Ruralité_Com < 200 m habts,0.001445
Urbanité Ruralité_Com > 200 m habts,0.002211
Urbanité Ruralité_Com rurale < 2 000 m habts,0.017743
Nb propriétaire,0.018298
Indice Démographique,0.018820
Pop 30-44 ans en 2014 (princ),0.019489
Pop 15 ans ou plus Autres en 2014 (compl),0.019730
"Pop 15 ans ou plus Artisans, Comm., Chefs entr. en 2014 (compl)",0.019874


In [61]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(height=1000, width=1500, showlegend = False, margin = {'l': 50}, ) # to avoid cropping of column names
fig.show()

#PCA on 30 Best features 

In [62]:
from sklearn.decomposition import PCA

# Instanciate PCA with 3 components
pca = PCA(n_components=3)

# Fit transform X_std_train
X_opt_train = pca.fit_transform(X_train)

# Apply on X_std_test
X_opt_test = pca.transform(X_test)

In [63]:
PC1 = X_opt_train[:, 0]
PC2 = X_opt_train[:, 1]
PC3 = X_opt_train[:, 2]

# Convert PC into a DataFrame
PC = pd.DataFrame(data=X_opt_train, columns=["PC1", "PC2", "PC3"])
# PC Head
PC.head()

,PC1,PC2,PC3
0,-0.206114,2.337405,-0.781127
1,-0.862863,0.686518,2.379402
2,-0.816904,0.627457,0.507799
3,-0.751091,-1.867085,-0.041542
4,-0.668346,-0.301329,-1.018060


In [74]:
PC.shape

(27808, 3)

In [64]:
# Use pca.explained_variance_ratio_
print("Explained Variance ration per PC: {}".format(pca.explained_variance_ratio_))
print("Total explained variance ratio: {}%".format(pca.explained_variance_ratio_.sum()))

Explained Variance ration per PC: [0.57434712 0.08276086 0.06119349]
Total explained variance ratio: 0.7183014696927544%


In [69]:
rf = RandomForestRegressor(max_depth = 20, min_samples_leaf = 2, min_samples_split= 4, n_estimators= 100)

In [71]:
# Fit the RF bestfeatures on the train set where the PCA was applied and checkout the score on the test
rf.fit(X_opt_train, y_train)
# Print R^2 scores
print("R2 score on training set fit on PCA: ", rf.score(X_opt_train, y_train))
print("R2 score on test set fit on PCA: ",     rf.score(X_opt_test, y_test))

R2 score on training set fit on PCA:  0.5916847769489051
R2 score on test set fit on PCA:  0.12977413477139044


In [ ]:
#Les scores du RF ne sont pas amélioré dans l'espace de la 
#PCA par rapport à l'espace de départ à 30 dimensions